# 計算グラフを設計

## CNNの計算グラフ
URL: http://www.ie110704.net/2017/10/24/tensorflow%E3%81%A7%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%80%81cnn%E3%82%92%E5%AE%9F%E8%A3%85%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F/

Tensorflow API: https://qiita.com/piyo_papa/items/1fca6138645a29a31245

In [1]:
import numpy as np
import tensorflow as tf

In [4]:
# 計算グラフ

# プレースホルダー
x_ = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y_ = tf.placeholder(tf.float32, shape=(None, 10))

# 畳み込み層1
with tf.name_scope('conv1'):
    conv1_features = 20 # 畳み込み層1の出力次元数
    max_pool_size1 = 2 # 畳み込み層1のマックスプーリングサイズ
    conv1_w = tf.Variable(tf.truncated_normal([5, 5, 1, conv1_features], stddev=0.1), dtype=tf.float32, name='conv1_w') # 畳み込み層1の重み(初期値)
    conv1_b = tf.Variable(tf.constant(0.1, shape=[conv1_features]), dtype=tf.float32, name='conv1_b') # 畳み込み層1のバイアス(初期値)
    conv1_c2 = tf.nn.conv2d(x_, conv1_w, strides=[1, 1, 1, 1], padding="SAME", name='conv1_conv2d') # 畳み込み層1-畳み込み
    conv1_relu = tf.nn.relu(conv1_c2+conv1_b, name='conv1_ReLU') # 畳み込み層1-ReLU
    conv1_mp = tf.nn.max_pool(conv1_relu, ksize=[1, max_pool_size1, max_pool_size1, 1], strides=[1, max_pool_size1, max_pool_size1, 1], padding="SAME", name='conv1_max_polling') #畳み込み層1-マックスプーリング

# 畳み込み層2
with tf.name_scope('conv2'):
    conv2_features = 50 # 畳み込み層2の出力次元数
    max_pool_size2 = 2 # 畳み込み層2のマックスプーリングのサイズ
    conv2_w = tf.Variable(tf.truncated_normal([5, 5, conv1_features, conv2_features], stddev=0.1), dtype=tf.float32, name='conv2_w') # 畳み込み層2の重み
    conv2_b = tf.Variable(tf.constant(0.1, shape=[conv2_features]), dtype=tf.float32, name='conv2_b') # 畳み込み層2のバイアス
    conv2_c2 = tf.nn.conv2d(conv1_mp, conv2_w, strides=[1, 1, 1, 1], padding="SAME", name='conv2_conv2d') # 畳み込み層2-畳み込み
    conv2_relu = tf.nn.relu(conv2_c2+conv2_b, name='conv2_ReLU') # 畳み込み層2-ReLU
    conv2_mp = tf.nn.max_pool(conv2_relu, ksize=[1, max_pool_size2, max_pool_size2, 1], strides=[1, max_pool_size2, max_pool_size2, 1], padding="SAME", name='conv2_max_polling') # 畳み込み層2-マックスプーリング

# 全結合層1
with tf.name_scope('fully1'):
    result_w = x_.shape[1] // (max_pool_size1*max_pool_size2)
    result_h = x_.shape[2] // (max_pool_size1*max_pool_size2)
    fc_input_size = result_w * result_h * conv2_features # 畳み込んだ結果、全結合層に入力する次元数
    fc_features = 500 # 全結合層の出力次元数（隠れ層の次元数）
    s = conv2_mp.get_shape().as_list() # [None, result_w, result_h, conv2_features]
    conv_result = tf.reshape(conv2_mp, [-1, s[1]*s[2]*s[3]]) # 畳み込みの結果を1*N層に変換
    fc1_w = tf.Variable(tf.truncated_normal([fc_input_size.value, fc_features], stddev=0.1), dtype=tf.float32, name='fully1_w') # 重み
    fc1_b = tf.Variable(tf.constant(0.1, shape=[fc_features]), dtype=tf.float32, name='fully1_b') # バイアス
    fc1 = tf.nn.relu(tf.matmul(conv_result, fc1_w)+fc1_b, name='fully1_ReLU') # 全結合層1

# 全結合層2
with tf.name_scope('fully2'):
    fc2_w = tf.Variable(tf.truncated_normal([fc_features, fc_features], stddev=0.1), dtype=tf.float32, name='fully2_w') # 重み
    fc2_b = tf.Variable(tf.constant(0.1, shape=[fc_features]), dtype=tf.float32, name='fully2_b') # バイアス
    fc2 = tf.nn.relu(tf.matmul(fc1, fc2_w)+fc2_b, name='fully1_ReLU') # 全結合層2

# 全結合層3
with tf.name_scope('fully3'):
    fc3_w = tf.Variable(tf.truncated_normal([fc_features, 10], stddev=0.1), dtype=tf.float32, name='fully3_w') # 重み
    fc3_b = tf.Variable(tf.constant(0.1, shape=[10]), dtype=tf.float32, name='fully3_b') # バイアス
    y = tf.matmul(fc2, fc3_w)+fc3_b

# クロスエントロピー誤差(改訂版)
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))
# 勾配法
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


# TensorBoardで追跡する変数を定義
file_writer = tf.summary.FileWriter("logsdata/CNN_graph", tf.get_default_graph())
file_writer.close()


## inverterの計算グラフ設計

In [1]:
import numpy as np
import tensorflow as tf

# 計算グラフ
g = tf.Graph()
with g.as_default():
    # プレースホルダー
    x_ = tf.placeholder(tf.float32, shape=(None, 128, 128, 3))
    y_ = tf.placeholder(tf.float32, shape=(None, 100))
    keep_prob = tf.placeholder(tf.float32)
    print(x_)
    print(y_)
    print(keep_prob)
    
    # 畳み込み層1
    with tf.name_scope('conv1'):
        conv1_features = 40 # 畳み込み層1の出力次元数
        max_pool_size1 = 2 # 畳み込み層1のマックスプーリングサイズ
        conv1_w = tf.Variable(tf.truncated_normal([5, 5, 3, conv1_features], stddev=0.1), dtype=tf.float32, name='conv1_w') # 畳み込み層1の重み(初期値)
        conv1_b = tf.Variable(tf.constant(0.1, shape=[conv1_features]), dtype=tf.float32, name='conv1_b') # 畳み込み層1のバイアス(初期値)
        conv1_c2 = tf.nn.conv2d(x_, conv1_w, strides=[1, 1, 1, 1], padding="SAME", name='conv1_conv2d') # 畳み込み層1-畳み込み
        conv1_relu = tf.nn.relu(conv1_c2+conv1_b, name='conv1_ReLU') #畳み込み層1-ReLU
        conv1_drop = tf.nn.dropout(conv1_relu, keep_prob, name='conv1_dropout')#畳み込み層1-ドロップアウト
        conv1_mp = tf.nn.max_pool(conv1_drop, ksize=[1, max_pool_size1, max_pool_size1, 1], strides=[1, max_pool_size1, max_pool_size1, 1], padding="SAME", name='conv1_max_polling') #畳み込み層1-マックスプーリング

    # 畳み込み層2
    with tf.name_scope('conv2'):
        conv2_features = 80 # 畳み込み層2の出力次元数
        max_pool_size2 = 2 # 畳み込み層2のマックスプーリングのサイズ
        conv2_w = tf.Variable(tf.truncated_normal([5, 5, conv1_features, conv2_features], stddev=0.1), dtype=tf.float32, name='conv2_w') # 畳み込み層2の重み
        conv2_b = tf.Variable(tf.constant(0.1, shape=[conv2_features]), dtype=tf.float32, name='conv2_b') # 畳み込み層2のバイアス
        conv2_c2 = tf.nn.conv2d(conv1_mp, conv2_w, strides=[1, 1, 1, 1], padding="SAME", name='conv2_conv2d') # 畳み込み層2-畳み込み
        conv2_relu = tf.nn.relu(conv2_c2+conv2_b, name='conv2_ReLU') # 畳み込み層2-ReLU
        conv2_drop = tf.nn.dropout(conv2_relu, keep_prob, name='conv2_dropout')#畳み込み層2-ドロップアウト
        conv2_mp = tf.nn.max_pool(conv2_drop, ksize=[1, max_pool_size2, max_pool_size2, 1], strides=[1, max_pool_size2, max_pool_size2, 1], padding="SAME", name='conv2_max_polling') # 畳み込み層2-マックスプーリング

    # 畳み込み層3
    with tf.name_scope('conv3'):
        conv3_features = 160 # 畳み込み層3の出力次元数
        max_pool_size3 = 2 # 畳み込み層3のマックスプーリングのサイズ
        conv3_w = tf.Variable(tf.truncated_normal([5, 5, conv2_features, conv3_features], stddev=0.1), dtype=tf.float32, name='conv3_w') # 畳み込み層3の重み
        conv3_b = tf.Variable(tf.constant(0.1, shape=[conv3_features]), dtype=tf.float32, name='conv3_b') # 畳み込み層3のバイアス
        conv3_c2 = tf.nn.conv2d(conv2_mp, conv3_w, strides=[1, 1, 1, 1], padding="SAME", name='conv3_conv2d') # 畳み込み層3-畳み込み
        conv3_relu = tf.nn.relu(conv3_c2+conv3_b, name='conv3_ReLU') # 畳み込み層3-ReLU
        conv3_drop = tf.nn.dropout(conv3_relu, keep_prob, name='conv3_dropout')#畳み込み層3-ドロップアウト
        conv3_mp = tf.nn.max_pool(conv3_drop, ksize=[1, max_pool_size3, max_pool_size3, 1], strides=[1, max_pool_size3, max_pool_size3, 1], padding="SAME", name='conv3_max_polling') # 畳み込み層3-マックスプーリング
    
    # 畳み込み層4
    with tf.name_scope('conv4'):
        conv4_features = 320 # 畳み込み層4の出力次元数
        max_pool_size4 = 2 # 畳み込み層4のマックスプーリングのサイズ
        conv4_w = tf.Variable(tf.truncated_normal([5, 5, conv3_features, conv4_features], stddev=0.1), dtype=tf.float32, name='conv4_w') # 畳み込み層4の重み
        conv4_b = tf.Variable(tf.constant(0.1, shape=[conv4_features]), dtype=tf.float32, name='conv4_b') # 畳み込み層4のバイアス
        conv4_c2 = tf.nn.conv2d(conv3_mp, conv4_w, strides=[1, 1, 1, 1], padding="SAME", name='conv4_conv2d') # 畳み込み層4-畳み込み
        conv4_relu = tf.nn.relu(conv4_c2+conv4_b, name='conv4_ReLU') # 畳み込み層4-ReLU
        conv4_drop = tf.nn.dropout(conv4_relu, keep_prob, name='conv4_dropout')#畳み込み層4-ドロップアウト
        conv4_mp = tf.nn.max_pool(conv4_drop, ksize=[1, max_pool_size4, max_pool_size4, 1], strides=[1, max_pool_size4, max_pool_size4, 1], padding="SAME", name='conv4_max_polling') # 畳み込み層4-マックスプーリング

    # 畳み込み層5
    with tf.name_scope('conv5'):
        conv5_features = 640 # 畳み込み層5の出力次元数
        max_pool_size5 = 2 # 畳み込み層5のマックスプーリングのサイズ
        conv5_w = tf.Variable(tf.truncated_normal([5, 5, conv4_features, conv5_features], stddev=0.1), dtype=tf.float32, name='conv5_w') # 畳み込み層5の重み
        conv5_b = tf.Variable(tf.constant(0.1, shape=[conv5_features]), dtype=tf.float32, name='conv5_b') # 畳み込み層5のバイアス
        conv5_c2 = tf.nn.conv2d(conv4_mp, conv5_w, strides=[1, 1, 1, 1], padding="SAME", name='conv5_conv2d') # 畳み込み層5-畳み込み
        conv5_relu = tf.nn.relu(conv5_c2+conv5_b, name='conv5_ReLU') # 畳み込み層5-ReLU
        conv5_drop = tf.nn.dropout(conv5_relu, keep_prob, name='conv5_dropout')#畳み込み層5-ドロップアウト
        conv5_mp = tf.nn.max_pool(conv5_drop, ksize=[1, max_pool_size5, max_pool_size5, 1], strides=[1, max_pool_size5, max_pool_size5, 1], padding="SAME", name='conv5_max_polling') # 畳み込み層5-マックスプーリング

    # 畳み込み層6
    with tf.name_scope('conv6'):
        conv6_features = 1280 # 畳み込み層6の出力次元数
        max_pool_size6 = 2 # 畳み込み層6のマックスプーリングのサイズ
        conv6_w = tf.Variable(tf.truncated_normal([5, 5, conv5_features, conv6_features], stddev=0.1), dtype=tf.float32, name='conv6_w') # 畳み込み層6の重み
        conv6_b = tf.Variable(tf.constant(0.1, shape=[conv6_features]), dtype=tf.float32, name='conv6_b') # 畳み込み層6のバイアス
        conv6_c2 = tf.nn.conv2d(conv5_mp, conv6_w, strides=[1, 1, 1, 1], padding="SAME", name='conv6_conv2d') # 畳み込み層6-畳み込み
        conv6_relu = tf.nn.relu(conv6_c2+conv6_b, name='conv6_ReLU') # 畳み込み層6-ReLU
        conv6_drop = tf.nn.dropout(conv6_relu, keep_prob, name='conv6_dropout')#畳み込み層6-ドロップアウト
        conv6_mp = tf.nn.max_pool(conv6_drop, ksize=[1, max_pool_size6, max_pool_size6, 1], strides=[1, max_pool_size6, max_pool_size6, 1], padding="SAME", name='conv6_max_polling') # 畳み込み層6-マックスプーリング
        
    
    # 全結合層1
    with tf.name_scope('fully1'):
        result_w = x_.shape[1] // (max_pool_size1*max_pool_size2*max_pool_size3*max_pool_size4*max_pool_size5*max_pool_size6)
        result_h = x_.shape[2] // (max_pool_size1*max_pool_size2*max_pool_size3*max_pool_size4*max_pool_size5*max_pool_size6)
        fc_input_size = result_w * result_h * conv6_features # 畳み込んだ結果、全結合層に入力する次元数
        fc_features = 1000 # 全結合層1の出力次元数（隠れ層の次元数）
        s = conv6_mp.get_shape().as_list() # [None, result_w, result_h, conv4_features]
        conv_result = tf.reshape(conv6_mp, [-1, s[1]*s[2]*s[3]]) # 畳み込みの結果を1*N層に変換
        fc1_w = tf.Variable(tf.truncated_normal([fc_input_size.value, fc_features], stddev=0.1), dtype=tf.float32, name='fully1_w') # 重み
        fc1_b = tf.Variable(tf.constant(0.1, shape=[fc_features]), dtype=tf.float32, name='fully1_b') # バイアス
        fc1 = tf.nn.relu(tf.matmul(conv_result, fc1_w)+fc1_b, name='fully1_ReLU') # 全結合層1
        fc1_drop = tf.nn.dropout(fc1, keep_prob, name='fully1_dropout') #ドロップアウト
    
    # 全結合層2
    with tf.name_scope('fully2'):
        fc2_features = 1000 # 全結合層2の出力次元数（隠れ層の次元数）
        fc2_w = tf.Variable(tf.truncated_normal([fc_features, fc2_features], stddev=0.1), dtype=tf.float32, name='fully2_w') # 重み
        fc2_b = tf.Variable(tf.constant(0.1, shape=[fc2_features]), dtype=tf.float32, name='fully2_b') # バイアス
        fc2 = tf.nn.relu(tf.matmul(fc1_drop, fc2_w)+fc2_b, name='fully2_ReLU') # 全結合層2
        fc2_drop = tf.nn.dropout(fc2, keep_prob, name='fully2_dropout')#ドロップアウト

    # 全結合層3
    with tf.name_scope('fully3'):
        fc3_features = 500 # 全結合層3の出力次元数（隠れ層の次元数）
        fc3_w = tf.Variable(tf.truncated_normal([fc2_features, fc3_features], stddev=0.1), dtype=tf.float32, name='fully3_w') # 重み
        fc3_b = tf.Variable(tf.constant(0.1, shape=[fc3_features]), dtype=tf.float32, name='fully3_b') # バイアス
        fc3 = tf.nn.relu(tf.matmul(fc2_drop, fc3_w)+fc3_b, name='fully3_ReLU') # 全結合層3
        fc3_drop = tf.nn.dropout(fc3, keep_prob, name='fully3_dropout')#ドロップアウト
    
    # 全結合層4
    with tf.name_scope('fully4'):
        fc4_features = 100 # 全結合層3の出力次元数（隠れ層の次元数）
        fc4_w = tf.Variable(tf.truncated_normal([fc3_features, fc4_features], stddev=0.1), dtype=tf.float32, name='fully4_w') # 重み
        fc4_b = tf.Variable(tf.constant(0.1, shape=[fc4_features]), dtype=tf.float32, name='fully4_b') # バイアス
        y = tf.matmul(fc3_drop, fc4_w)+fc4_b
        print(y)

    # 平均2乗和誤差(square_error)
    with tf.name_scope('loss'):
        loss = tf.reduce_sum(tf.square(y_ - y), name='loss')
        print(loss)
    
    # 勾配法
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
        print(train_step)

    # グラフを保存する．
    #saver = tf.train.Saver()
 
    """# TensorBoardで追跡する変数を定義
    file_writer = tf.summary.FileWriter("logsdata/inverter_graph", tf.get_default_graph())
    file_writer.close()"""

Tensor("Placeholder:0", shape=(?, 128, 128, 3), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 100), dtype=float32)
Tensor("Placeholder_2:0", dtype=float32)
Tensor("fully4/add:0", shape=(?, 100), dtype=float32)
Tensor("loss/loss:0", shape=(), dtype=float32)
name: "train/Adam"
op: "NoOp"
input: "^train/Adam/update_conv1/conv1_w/ApplyAdam"
input: "^train/Adam/update_conv1/conv1_b/ApplyAdam"
input: "^train/Adam/update_conv2/conv2_w/ApplyAdam"
input: "^train/Adam/update_conv2/conv2_b/ApplyAdam"
input: "^train/Adam/update_conv3/conv3_w/ApplyAdam"
input: "^train/Adam/update_conv3/conv3_b/ApplyAdam"
input: "^train/Adam/update_conv4/conv4_w/ApplyAdam"
input: "^train/Adam/update_conv4/conv4_b/ApplyAdam"
input: "^train/Adam/update_conv5/conv5_w/ApplyAdam"
input: "^train/Adam/update_conv5/conv5_b/ApplyAdam"
input: "^train/Adam/update_conv6/conv6_w/ApplyAdam"
input: "^train/Adam/update_conv6/conv6_b/ApplyAdam"
input: "^train/Adam/update_fully1/fully1_w/ApplyAdam"
input: "^train/Adam/update_ful

### データ読み込み&データ生成(バッチによる読み込み)

In [13]:
import numpy as np

i = 28901
rec1 = np.load('/opt/pfw/dragon_ball_reconsted/reconsted/disc11/story1/img_'+str(i)+'_reconsted.npy')
z1 = np.load('/opt/pfw/dragon_ball_reconsted/z/disc11/story1/img_'+str(i)+'_z.npy')
print(rec1.shape)
print(z1.shape)
#Noneがあるので，4階テンソルでも大丈夫

(1, 128, 128, 3)
(1, 100)


In [6]:
#Noneがあるので，1つのファイルが4階テンソルでも大丈夫
import numpy as np
import random
import os
import glob
import time

start = time.time()

batch_size = 60

#再構成した画像のPATHのリスト
path_rec = "/opt/pfw/dragon_ball_reconsted/reconsted/disc11/story1/*.npy"
filelists_rec = glob.glob(path_rec, recursive=True)
print(len(filelists_rec))

#zのPATHのリスト
path_z = "/opt/pfw/dragon_ball_reconsted/z/disc11/story1/*.npy"
filelists_z = glob.glob(path_z, recursive=True)
print(len(filelists_z))

#サンプルしてくるインデックスのリスト
ind = random.sample(range(len(filelists_rec)), batch_size)
#print("index\n%s\n" %ind)

#サンプルしてくるインデックスの各PATHを格納したリスト
batch_images_rec = [filelists_rec[i] for i in ind ]
batch_images_z = [filelists_z[i] for i in ind ]

#再構成した画像とzの各バッチ
batch_x = np.array([np.load(f).tolist() for f in batch_images_rec]) #再構成画像のファイル読み込み
batch_z = np.array([np.load(f).tolist() for f in batch_images_z]) #zのファイル読み込み

print(np.array(batch_x).shape)
print(np.array(batch_x).shape)

print(np.array(batch_z).shape)
print(np.array(batch_z).shape)

print(time.time()-start)

22711
22711
(60, 128, 128, 3)
(60, 128, 128, 3)
(60, 100)
(60, 100)
4.973942518234253


### データ読み込み&データ生成(バッチによる読み込み)の関数

In [1]:
def batch(batch_size):
    #batch_size = 60

    #再構成した画像のPATHのリスト
    path_rec = "/opt/pfw/dragon_ball_reconsted/reconsted/disc11/story1/*.npy"
    filelists_rec = glob.glob(path_rec, recursive=True)

    #zのPATHのリスト
    path_z = "/opt/pfw/dragon_ball_reconsted/z/disc11/story1/*.npy"
    filelists_z = glob.glob(path_z, recursive=True)

    #サンプルしてくるインデックスのリスト
    ind = ra.sample(range(len(filelists_rec)), batch_size)

    #サンプルしてくるインデックスの各PATHを格納したリスト
    batch_images_rec = [filelists_rec[i] for i in ind ]
    batch_images_z = [filelists_z[i] for i in ind ]

    #再構成した画像とzの各バッチ
    batch_x = np.array([np.load(f).tolist() for f in batch_images_rec]) #再構成画像のファイル読み込み
    batch_z = np.array([np.load(f).tolist() for f in batch_images_z]) #zのファイル読み込み
    
    return batch_x, batch_z

### inverter実行フェーズ

In [ ]:
import tensorflow as tf
import numpy as np
import random as ra
import os
import glob
from numpy.random import *
import time

start = time.time()

#実行フェーズ
n_epochs = 50000000
training_loss = []
eps = 0.01
batch_size = 60
batch_x, batch_z = batch(batch_size)

with tf.Session(graph=g) as sess:
    
    #変数を初期化
    sess.run(tf.global_variables_initializer())
    
    # TensorBoardで追跡する変数を定義
    with tf.name_scope('summary'):
        tf.summary.scalar('loss', loss)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logsdata/inverter_graph_try', sess.graph)
        print(merged)
    
    
    #500エポックでモデルをトレーニング
    for e in range(n_epochs):
        c, _, summary = sess.run([loss, train_step, merged], feed_dict={x_: batch_x, y_: batch_z, keep_prob: 0.5})
        writer.add_summary(summary, e)
        training_loss.append(c)
        
        print('Epoch %4d: %.4f' % (e, c), '\t', 'time %4d' %(time.time()-start))
        
        if not e % 50:
            batch_x, batch_z = batch(batch_size)
        
        if(e > 0 and c < eps):
            saver.save(sess, './savepoint/inverter_graph_try/train_epoch_'+str(e)+'/trained-model')
            break
        
        if(e % 1000 == 0):
            saver.save(sess, './savepoint/inverter_graph_try/train_epoch_'+str(e)+'/trained-model')

Tensor("summary/Merge/MergeSummary:0", shape=(), dtype=string)
Epoch    0: 16134639500394496.0000 	 time   15
Epoch    1: 9326594497708032.0000 	 time   61
Epoch    2: 5788967374946304.0000 	 time   68
Epoch    3: 3654511168061440.0000 	 time   75


### 学習途中からstartする方法(途中から学習できるが，tensorboardに記録がつかない)

In [ ]:
import tensorflow as tf
import numpy as np
import random as ra
import os
import glob
from numpy.random import *
import time

start = time.time()

n_epochs = 50000000
training_loss = []
eps = 0.01
batch_size = 60
batch_x, batch_z = batch(batch_size)
kp_p = 1.0

epo = 23000

g2 = tf.Graph()
with tf.Session(graph=g2) as sess:
    #計算グラフの復元
    new_saver = tf.train.import_meta_graph('./savepoint/inverter_graph/train_epoch_'+str(epo)+'/trained-model.meta')
    #実行フェーズでの値の復元
    new_saver.restore(sess, './savepoint/inverter_graph/train_epoch_'+str(epo)+'/trained-model')
    
    loss = sess.run('loss/loss:0', feed_dict={'Placeholder:0' : batch_x, 'Placeholder_1:0' : batch_z, 'Placeholder_2:0' : kp_p})
    train_step = sess.run('train/Adam', feed_dict={'Placeholder:0' : batch_x, 'Placeholder_1:0' : batch_z, 'Placeholder_2:0' : kp_p})
    print(loss)
    print(train_step)
    
    # グラフを保存する．
    saver = tf.train.Saver()
    
    # TensorBoardで追跡する変数を定義
    with tf.name_scope('summary'):
        tf.summary.scalar('loss', loss)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logsdata/inverter_graph_try2', sess.graph)
    
    #500エポックでモデルをトレーニング
    for e in range(n_epochs):
        e = epo+e+1
        c, _, summary = sess.run(['loss/loss:0', 'train/Adam', merged], feed_dict={'Placeholder:0' : batch_x, 'Placeholder_1:0' : batch_z, 'Placeholder_2:0' : 0.5})
        writer.add_summary(summary, e)
        training_loss.append(c)
        
        print('Epoch %4d: %.4f' % (e, c), '\t', 'time %4d' %(time.time()-start))
        
        if not e % 50:
            batch_x, batch_z = batch(batch_size)
        
        if(e > 0 and c < eps):
            saver.save(sess, './savepoint/inverter_graph_try2/train_epoch_'+str(e)+'/trained-model')
            break
        
        if(e % 1000 == 0):
            saver.save(sess, './savepoint/inverter_graph_try2/train_epoch_'+str(e)+'/trained-model')

INFO:tensorflow:Restoring parameters from ./savepoint/inverter_graph/train_epoch_23000/trained-model
8037.0073
None
Epoch 23001: 8194.6553 	 time   23
Epoch 23002: 8216.8291 	 time   32
Epoch 23003: 8266.0254 	 time   40
Epoch 23004: 8249.9365 	 time   51
Epoch 23005: 8214.6309 	 time   60
Epoch 23006: 8213.3750 	 time   67
Epoch 23007: 8240.7812 	 time   76
Epoch 23008: 8221.1699 	 time   84
Epoch 23009: 8239.7012 	 time   93
Epoch 23010: 8235.5342 	 time  102
Epoch 23011: 8235.9404 	 time  111
Epoch 23012: 8260.5195 	 time  121
Epoch 23013: 8226.6592 	 time  131
Epoch 23014: 8265.2041 	 time  143
Epoch 23015: 8215.2227 	 time  153


### 学習途中からstartする方法(計算グラフを持ってきて，tensorboardに記録をつける．)

In [ ]:
import tensorflow as tf
import numpy as np
import random as ra
import os
import glob
from numpy.random import *
import time

# 計算グラフ
g2 = tf.Graph()
with g2.as_default():
    # プレースホルダー
    x_ = tf.placeholder(tf.float32, shape=(None, 128, 128, 3))
    y_ = tf.placeholder(tf.float32, shape=(None, 100))
    keep_prob = tf.placeholder(tf.float32)

    # 畳み込み層1
    with tf.name_scope('conv1'):
        conv1_features = 40 # 畳み込み層1の出力次元数
        max_pool_size1 = 2 # 畳み込み層1のマックスプーリングサイズ
        conv1_w = tf.Variable(tf.truncated_normal([5, 5, 3, conv1_features], stddev=0.1), dtype=tf.float32, name='conv1_w') # 畳み込み層1の重み(初期値)
        conv1_b = tf.Variable(tf.constant(0.1, shape=[conv1_features]), dtype=tf.float32, name='conv1_b') # 畳み込み層1のバイアス(初期値)
        conv1_c2 = tf.nn.conv2d(x_, conv1_w, strides=[1, 1, 1, 1], padding="SAME", name='conv1_conv2d') # 畳み込み層1-畳み込み
        conv1_relu = tf.nn.relu(conv1_c2+conv1_b, name='conv1_ReLU') #畳み込み層1-ReLU
        conv1_drop = tf.nn.dropout(conv1_relu, keep_prob, name='conv1_dropout')#畳み込み層1-ドロップアウト
        conv1_mp = tf.nn.max_pool(conv1_drop, ksize=[1, max_pool_size1, max_pool_size1, 1], strides=[1, max_pool_size1, max_pool_size1, 1], padding="SAME", name='conv1_max_polling') #畳み込み層1-マックスプーリング

    # 畳み込み層2
    with tf.name_scope('conv2'):
        conv2_features = 80 # 畳み込み層2の出力次元数
        max_pool_size2 = 2 # 畳み込み層2のマックスプーリングのサイズ
        conv2_w = tf.Variable(tf.truncated_normal([5, 5, conv1_features, conv2_features], stddev=0.1), dtype=tf.float32, name='conv2_w') # 畳み込み層2の重み
        conv2_b = tf.Variable(tf.constant(0.1, shape=[conv2_features]), dtype=tf.float32, name='conv2_b') # 畳み込み層2のバイアス
        conv2_c2 = tf.nn.conv2d(conv1_mp, conv2_w, strides=[1, 1, 1, 1], padding="SAME", name='conv2_conv2d') # 畳み込み層2-畳み込み
        conv2_relu = tf.nn.relu(conv2_c2+conv2_b, name='conv2_ReLU') # 畳み込み層2-ReLU
        conv2_drop = tf.nn.dropout(conv2_relu, keep_prob, name='conv2_dropout')#畳み込み層2-ドロップアウト
        conv2_mp = tf.nn.max_pool(conv2_drop, ksize=[1, max_pool_size2, max_pool_size2, 1], strides=[1, max_pool_size2, max_pool_size2, 1], padding="SAME", name='conv2_max_polling') # 畳み込み層2-マックスプーリング

    # 畳み込み層3
    with tf.name_scope('conv3'):
        conv3_features = 160 # 畳み込み層3の出力次元数
        max_pool_size3 = 2 # 畳み込み層3のマックスプーリングのサイズ
        conv3_w = tf.Variable(tf.truncated_normal([5, 5, conv2_features, conv3_features], stddev=0.1), dtype=tf.float32, name='conv3_w') # 畳み込み層3の重み
        conv3_b = tf.Variable(tf.constant(0.1, shape=[conv3_features]), dtype=tf.float32, name='conv3_b') # 畳み込み層3のバイアス
        conv3_c2 = tf.nn.conv2d(conv2_mp, conv3_w, strides=[1, 1, 1, 1], padding="SAME", name='conv3_conv2d') # 畳み込み層3-畳み込み
        conv3_relu = tf.nn.relu(conv3_c2+conv3_b, name='conv3_ReLU') # 畳み込み層3-ReLU
        conv3_drop = tf.nn.dropout(conv3_relu, keep_prob, name='conv3_dropout')#畳み込み層3-ドロップアウト
        conv3_mp = tf.nn.max_pool(conv3_drop, ksize=[1, max_pool_size3, max_pool_size3, 1], strides=[1, max_pool_size3, max_pool_size3, 1], padding="SAME", name='conv3_max_polling') # 畳み込み層3-マックスプーリング
    
    # 畳み込み層4
    with tf.name_scope('conv4'):
        conv4_features = 320 # 畳み込み層4の出力次元数
        max_pool_size4 = 2 # 畳み込み層4のマックスプーリングのサイズ
        conv4_w = tf.Variable(tf.truncated_normal([5, 5, conv3_features, conv4_features], stddev=0.1), dtype=tf.float32, name='conv4_w') # 畳み込み層4の重み
        conv4_b = tf.Variable(tf.constant(0.1, shape=[conv4_features]), dtype=tf.float32, name='conv4_b') # 畳み込み層4のバイアス
        conv4_c2 = tf.nn.conv2d(conv3_mp, conv4_w, strides=[1, 1, 1, 1], padding="SAME", name='conv4_conv2d') # 畳み込み層4-畳み込み
        conv4_relu = tf.nn.relu(conv4_c2+conv4_b, name='conv4_ReLU') # 畳み込み層4-ReLU
        conv4_drop = tf.nn.dropout(conv4_relu, keep_prob, name='conv4_dropout')#畳み込み層4-ドロップアウト
        conv4_mp = tf.nn.max_pool(conv4_drop, ksize=[1, max_pool_size4, max_pool_size4, 1], strides=[1, max_pool_size4, max_pool_size4, 1], padding="SAME", name='conv4_max_polling') # 畳み込み層4-マックスプーリング

    # 畳み込み層5
    with tf.name_scope('conv5'):
        conv5_features = 640 # 畳み込み層5の出力次元数
        max_pool_size5 = 2 # 畳み込み層5のマックスプーリングのサイズ
        conv5_w = tf.Variable(tf.truncated_normal([5, 5, conv4_features, conv5_features], stddev=0.1), dtype=tf.float32, name='conv5_w') # 畳み込み層5の重み
        conv5_b = tf.Variable(tf.constant(0.1, shape=[conv5_features]), dtype=tf.float32, name='conv5_b') # 畳み込み層5のバイアス
        conv5_c2 = tf.nn.conv2d(conv4_mp, conv5_w, strides=[1, 1, 1, 1], padding="SAME", name='conv5_conv2d') # 畳み込み層5-畳み込み
        conv5_relu = tf.nn.relu(conv5_c2+conv5_b, name='conv5_ReLU') # 畳み込み層5-ReLU
        conv5_drop = tf.nn.dropout(conv5_relu, keep_prob, name='conv5_dropout')#畳み込み層5-ドロップアウト
        conv5_mp = tf.nn.max_pool(conv5_drop, ksize=[1, max_pool_size5, max_pool_size5, 1], strides=[1, max_pool_size5, max_pool_size5, 1], padding="SAME", name='conv5_max_polling') # 畳み込み層5-マックスプーリング

    # 畳み込み層6
    with tf.name_scope('conv6'):
        conv6_features = 1280 # 畳み込み層6の出力次元数
        max_pool_size6 = 2 # 畳み込み層6のマックスプーリングのサイズ
        conv6_w = tf.Variable(tf.truncated_normal([5, 5, conv5_features, conv6_features], stddev=0.1), dtype=tf.float32, name='conv6_w') # 畳み込み層6の重み
        conv6_b = tf.Variable(tf.constant(0.1, shape=[conv6_features]), dtype=tf.float32, name='conv6_b') # 畳み込み層6のバイアス
        conv6_c2 = tf.nn.conv2d(conv5_mp, conv6_w, strides=[1, 1, 1, 1], padding="SAME", name='conv6_conv2d') # 畳み込み層6-畳み込み
        conv6_relu = tf.nn.relu(conv6_c2+conv6_b, name='conv6_ReLU') # 畳み込み層6-ReLU
        conv6_drop = tf.nn.dropout(conv6_relu, keep_prob, name='conv6_dropout')#畳み込み層6-ドロップアウト
        conv6_mp = tf.nn.max_pool(conv6_drop, ksize=[1, max_pool_size6, max_pool_size6, 1], strides=[1, max_pool_size6, max_pool_size6, 1], padding="SAME", name='conv6_max_polling') # 畳み込み層6-マックスプーリング
        
    
    # 全結合層1
    with tf.name_scope('fully1'):
        result_w = x_.shape[1] // (max_pool_size1*max_pool_size2*max_pool_size3*max_pool_size4*max_pool_size5*max_pool_size6)
        result_h = x_.shape[2] // (max_pool_size1*max_pool_size2*max_pool_size3*max_pool_size4*max_pool_size5*max_pool_size6)
        fc_input_size = result_w * result_h * conv6_features # 畳み込んだ結果、全結合層に入力する次元数
        fc_features = 1000 # 全結合層1の出力次元数（隠れ層の次元数）
        s = conv6_mp.get_shape().as_list() # [None, result_w, result_h, conv4_features]
        conv_result = tf.reshape(conv6_mp, [-1, s[1]*s[2]*s[3]]) # 畳み込みの結果を1*N層に変換
        fc1_w = tf.Variable(tf.truncated_normal([fc_input_size.value, fc_features], stddev=0.1), dtype=tf.float32, name='fully1_w') # 重み
        fc1_b = tf.Variable(tf.constant(0.1, shape=[fc_features]), dtype=tf.float32, name='fully1_b') # バイアス
        fc1 = tf.nn.relu(tf.matmul(conv_result, fc1_w)+fc1_b, name='fully1_ReLU') # 全結合層1
        fc1_drop = tf.nn.dropout(fc1, keep_prob, name='fully1_dropout') #ドロップアウト
    
    # 全結合層2
    with tf.name_scope('fully2'):
        fc2_features = 1000 # 全結合層2の出力次元数（隠れ層の次元数）
        fc2_w = tf.Variable(tf.truncated_normal([fc_features, fc2_features], stddev=0.1), dtype=tf.float32, name='fully2_w') # 重み
        fc2_b = tf.Variable(tf.constant(0.1, shape=[fc2_features]), dtype=tf.float32, name='fully2_b') # バイアス
        fc2 = tf.nn.relu(tf.matmul(fc1_drop, fc2_w)+fc2_b, name='fully2_ReLU') # 全結合層2
        fc2_drop = tf.nn.dropout(fc2, keep_prob, name='fully2_dropout')#ドロップアウト

    # 全結合層3
    with tf.name_scope('fully3'):
        fc3_features = 500 # 全結合層3の出力次元数（隠れ層の次元数）
        fc3_w = tf.Variable(tf.truncated_normal([fc2_features, fc3_features], stddev=0.1), dtype=tf.float32, name='fully3_w') # 重み
        fc3_b = tf.Variable(tf.constant(0.1, shape=[fc3_features]), dtype=tf.float32, name='fully3_b') # バイアス
        fc3 = tf.nn.relu(tf.matmul(fc2_drop, fc3_w)+fc3_b, name='fully3_ReLU') # 全結合層3
        fc3_drop = tf.nn.dropout(fc3, keep_prob, name='fully3_dropout')#ドロップアウト
    
    # 全結合層4
    with tf.name_scope('fully4'):
        fc4_features = 100 # 全結合層3の出力次元数（隠れ層の次元数）
        fc4_w = tf.Variable(tf.truncated_normal([fc3_features, fc4_features], stddev=0.1), dtype=tf.float32, name='fully4_w') # 重み
        fc4_b = tf.Variable(tf.constant(0.1, shape=[fc4_features]), dtype=tf.float32, name='fully4_b') # バイアス
        y = tf.matmul(fc3_drop, fc4_w)+fc4_b

    # 平均2乗和誤差(square_error)
    with tf.name_scope('loss'):
        loss = tf.reduce_sum(tf.square(y_ - y), name='loss')
    
    # 勾配法
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    # グラフを保存する．
    saver = tf.train.Saver()

#実行フェーズ
start = time.time()

n_epochs = 50000000
training_loss = []
eps = 0.01
batch_size = 60
batch_x, batch_z = batch(batch_size)
kp_p = 1.0

epo = 23000

with tf.Session(graph=g2) as sess:
    #実行フェーズでの値の復元
    saver.restore(sess, './savepoint/inverter_graph/train_epoch_'+str(epo)+'/trained-model')
    
    """loss = sess.run('loss/loss:0', feed_dict={x_ : batch_x, y_ : batch_z, keep_prob : kp_p})
    train_step = sess.run('train/Adam', feed_dict={x_ : batch_x, y_ : batch_z, keep_prob : kp_p})
    print(loss)
    print(train_step)"""
    
    # TensorBoardで追跡する変数を定義
    with tf.name_scope('summary'):
        tf.summary.scalar('loss', loss)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logsdata/inverter_graph_try', sess.graph)
    
    #500エポックでモデルをトレーニング
    for e in range(n_epochs):
        c, _, summary = sess.run([loss, train_step, merged], feed_dict={x_: batch_x, y_: batch_z, keep_prob: 0.5})
        writer.add_summary(summary, e)
        training_loss.append(c)
        
        e = epo+e+1
        print('Epoch %4d: %.4f' % (e, c), '\t', 'time %4d' %(time.time()-start))
        
        if not e % 50:
            batch_x, batch_z = batch(batch_size)
        
        if(e > 0 and c < eps):
            saver.save(sess, './savepoint/inverter_graph_try/train_epoch_'+str(e)+'/trained-model')
            break
        
        if(e % 1000 == 0):
            saver.save(sess, './savepoint/inverter_graph_try/train_epoch_'+str(e)+'/trained-model')
        e = e-epo-1

INFO:tensorflow:Restoring parameters from ./savepoint/inverter_graph/train_epoch_23000/trained-model
Epoch 23001: 8522.3438 	 time   13
Epoch 23002: 8510.5488 	 time   20
Epoch 23003: 8530.4004 	 time   27
Epoch 23004: 8506.2061 	 time   34
Epoch 23005: 8484.5254 	 time   41
Epoch 23006: 8537.2529 	 time   48
Epoch 23007: 8526.0908 	 time   55
Epoch 23008: 8449.5430 	 time   61
Epoch 23009: 8526.8652 	 time   68
Epoch 23010: 8477.0176 	 time   75
Epoch 23011: 8535.7549 	 time   82
Epoch 23012: 8458.3369 	 time   89
Epoch 23013: 8498.4834 	 time   96
Epoch 23014: 8529.1543 	 time  103
Epoch 23015: 8470.9932 	 time  110
Epoch 23016: 8492.1816 	 time  117
Epoch 23017: 8578.6582 	 time  123
Epoch 23018: 8429.1543 	 time  130
Epoch 23019: 8489.9355 	 time  137
Epoch 23020: 8522.6553 	 time  143
Epoch 23021: 8498.9268 	 time  150
Epoch 23022: 8527.1602 	 time  157
Epoch 23023: 8462.2031 	 time  163
Epoch 23024: 8520.7246 	 time  170
Epoch 23025: 8475.8721 	 time  177
Epoch 23026: 8473.6738 	